## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Caravelas,BR,-17.7125,-39.2481,66.78,75,80,4.56,broken clouds
1,1,Alofi,NU,-19.0595,-169.9187,76.89,73,20,10.36,few clouds
2,2,Bethanien,NaN,-26.5020,17.1583,58.37,24,0,6.96,clear sky
3,3,Samarai,PG,-10.6167,150.6667,79.54,78,75,14.67,broken clouds
4,4,Hobart,AU,-42.8794,147.3294,45.27,88,75,1.01,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#David Comment: This is where things are inputted to start the min and max tempertures. I chose 75 and 90 moving forward.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Alofi,NU,-19.0595,-169.9187,76.89,73,20,10.36,few clouds
3,3,Samarai,PG,-10.6167,150.6667,79.54,78,75,14.67,broken clouds
7,7,Kapaa,US,22.0752,-159.3190,86.32,81,1,4.00,clear sky
9,9,Tra Vinh,VN,9.9347,106.3453,78.53,85,100,8.72,overcast clouds
12,12,Touros,BR,-5.1989,-35.4608,76.10,81,20,16.44,few clouds
...,...,...,...,...,...,...,...,...,...,...
433,433,Riachao,BR,-7.3619,-46.6172,76.33,29,12,4.03,few clouds
435,435,Kavieng,PG,-2.5744,150.7967,83.48,76,27,10.87,light rain
438,438,Sur,OM,22.5667,59.5289,87.98,58,62,10.38,broken clouds
443,443,El Pino,HN,15.7000,-86.9333,87.48,83,75,11.50,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
#David Comment: look for null with isnull function and sum all instances with the sum function
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Alofi,NU,76.89,few clouds,-19.0595,-169.9187,
3,Samarai,PG,79.54,broken clouds,-10.6167,150.6667,
7,Kapaa,US,86.32,clear sky,22.0752,-159.3190,
9,Tra Vinh,VN,78.53,overcast clouds,9.9347,106.3453,
12,Touros,BR,76.10,few clouds,-5.1989,-35.4608,
30,Port Blair,IN,81.21,overcast clouds,11.6667,92.7500,
36,La Ronge,CA,84.45,smoke,55.1001,-105.2842,
42,Hami,CN,76.10,clear sky,42.8000,93.4500,
45,Farah,AF,78.30,clear sky,32.5000,62.5000,
50,Panukulan,PH,84.06,overcast clouds,14.9333,121.8144,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
#David Comment: There were a lot of values that were just a space that did not get picked up in the isnull function, so I just nulled them
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
#David Comment: After changing all spaces to null, then we can drop them
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))